# 📓 01_player_sampling_and_stats.ipynb
**Purpose:** Load player data, inspect it, and prepare for performance modeling.

In [ ]:
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team

In [ ]:

calcutta_data = {
    "players": {},
    "tournaments": {},
    "teams": []
}

players = calcutta_data["players"]
tournaments = calcutta_data["tournaments"]
teams = calcutta_data["teams"]  # Optional global team structure if used

In [ ]:


golf_data_file = "golf_data.pkl"


def load_data(path=golf_data_file):
    global calcutta_data, players, tournaments, teams
    with open(path, "rb") as f:
        loaded = pickle.load(f)

    calcutta_data["players"] = loaded.get("players", {})
    calcutta_data["tournaments"] = loaded.get("tournaments", {})
    calcutta_data["teams"] = loaded.get("teams", [])

    # Rebind the globals for convenience
    players = calcutta_data["players"]
    tournaments = calcutta_data["tournaments"]
    teams = calcutta_data["teams"]

    print("✅ Data loaded successfully.")
    print(f"players: {len(players)}, teams: {len(teams)}, tournaments: {len(tournaments)}")

In [ ]:
load_data() 

In [ ]:
# --- Inspect Example Player ---
example_player = next(iter(players.values() if isinstance(players, dict) else players))

print(f"Example Player: {example_player.name}")
print(f"Total Rounds Recorded: {len(example_player.rounds)}")

# Print some details for the first few rounds
for i, round_info in enumerate(example_player.rounds[:5]):  # Show up to 5 rounds
    print(f" Round {i+1}:")
    print(f"  Tournament: {round_info.tournament_name}")
    print(f"  Round Number: {round_info.round_number}")
    print(f"  Handicap: {round_info.handicap}")
    print(f"  Tee: {round_info.tee}")
    print(f"  Net Score: {round_info.net}")
    print(f"  Total Gross Score: {round_info.total}")

In [ ]:
# --- Statistics ---

# Make sure players is either a list or dictionary
player_list = players.values() if isinstance(players, dict) else players

# Number of players
num_players = len(player_list)

# Number of rounds per player
rounds_per_player = [len(player.rounds) for player in player_list]

# All net scores and valid handicaps across all players and rounds
all_net_scores = []
all_handicaps = []

for player in player_list:
    for round_info in player.rounds:
        all_net_scores.append(round_info.net)
        if round_info.handicap is not None and not np.isnan(round_info.handicap):
            all_handicaps.append(round_info.handicap)

# Print statistics
print(f"\nStatistics:")
print(f"Total Players: {num_players}")
print(f"Average Rounds per Player: {np.mean(rounds_per_player):.2f}")
print(f"Min Rounds per Player: {np.min(rounds_per_player)}")
print(f"Max Rounds per Player: {np.max(rounds_per_player)}")

if all_net_scores:
    print(f"\nNet Score Statistics:")
    print(f"  Average Net Score: {np.mean(all_net_scores):.2f}")
    print(f"  Net Score Range: {np.min(all_net_scores)} to {np.max(all_net_scores)}")

if all_handicaps:
    print(f"\nHandicap Statistics:")
    print(f"  Average Handicap: {np.mean(all_handicaps):.2f}")
    print(f"  Handicap Range: {np.min(all_handicaps)} to {np.max(all_handicaps)}")
else:
    print("\n⚠️ Warning: No valid handicaps found.")


In [ ]:
# --- Plot Net Score Distribution ---
plt.hist(all_net_scores, bins=30, edgecolor='black')
plt.title('Distribution of Net Scores (All Players)')
plt.xlabel('Net Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# --- Plot Handicap Distribution ---
plt.hist(all_handicaps, bins=range(int(min(all_handicaps)) - 1, int(max(all_handicaps)) + 2), edgecolor='black')
plt.title('Distribution of Handicaps (All Players)')
plt.xlabel('Handicap')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# --- Optional: Plot rounds per player ---
plt.hist(rounds_per_player, bins=range(0, max(rounds_per_player)+5, 2), edgecolor='black')
plt.title('Distribution of Rounds per Player')
plt.xlabel('Number of Rounds')
plt.ylabel('Number of Players')
plt.grid(True)
plt.show()

In [ ]:
# --- Ready for Sampling Model ---

def sample_score_simple(player):
    return random.choice(player.historical_scores)

def sample_score_weighted(player, decay_factor=0.9):
    scores = player.historical_scores
    n = len(scores)
    weights = [decay_factor ** (n - i - 1) for i in range(n)]
    return random.choices(scores, weights=weights, k=1)[0]

In [ ]:
# --- Test Sampling ---
print("\n🧪 Sampling Test:")
print(f"Simple Sample for {example_player.name}: {sample_score_simple(example_player)}")
print(f"Weighted Sample for {example_player.name}: {sample_score_weighted(example_player)}")

In [ ]:
# --- Save player names (optional) ---
player_names = [p.name for p in (players.values() if isinstance(players, dict) else players)]

In [ ]:
# --- Print Top Players by Average Net Score with Fixed Columns ---

def calculate_and_print_player_net_stats(players, top_n=20, min_rounds=5):
    """
    Calculate and print top players based on average net score.
    
    Args:
        players: list or dict of Player objects
        top_n: number of top players to display
        min_rounds: minimum number of rounds a player must have to be considered
    """
    player_stats = []

    player_list = players.values() if isinstance(players, dict) else players

    for player in player_list:
        net_scores = [round_info.net for round_info in player.rounds if round_info.net is not None]
        if len(net_scores) >= min_rounds:
            avg_net = np.mean(net_scores)
            std_net = np.std(net_scores)
            num_rounds = len(net_scores)
            player_stats.append((player.name, avg_net, std_net, num_rounds))
    
    # Sort by lowest average net score
    player_stats.sort(key=lambda x: x[1])

    # --- Print ---
    header = "{:<5} {:<25} {:>15} {:>12} {:>10}".format("Rank", "Player Name", "Avg Net Score", "Std Dev", "Rounds")
    print(f"\n🏌️ Top {top_n} Players by Average Net Score (Minimum {min_rounds} Rounds):\n")
    print(header)
    print("-" * len(header))

    for idx, (name, avg_score, std_dev, num_rounds) in enumerate(player_stats[:top_n], start=1):
        row = "{:<5} {:<25} {:>15.2f} {:>12.2f} {:>10}".format(idx, name, avg_score, std_dev, num_rounds)
        print(row)

# --- Example Usage ---

calculate_and_print_player_net_stats(players, top_n=50, min_rounds=4)